In [8]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd
import folium

# 미술관 API 요청 URL
art_gallery_url = 'http://www.culture.go.kr/openapi/rest/cultureartspaces/artgallery'

# 박물관 API 요청 URL
museum_url = 'http://www.culture.go.kr/openapi/rest/cultureartspaces/museum'

# 공연장 API 요청 URL
perform_url = 'http://www.culture.go.kr/openapi/rest/cultureartspaces/performingplace'

# 필요한 파라미터 설정 (예시)
params = {
    'ServiceKey': 'J7eSd1v9CxPBqa+NuhF9JKvRQX2EHh9t6wtILRwPAk4hl/BvkK0gDocAeIFZ/yiMmMIFgN7vHDLR58rIdKh4bA==',  # 실제 서비스 키로 변경해야 합니다.
    'areaCode': 1,
}

# 미술관 정보 가져오기
response_gallery = requests.get(art_gallery_url, params=params)

if response_gallery.status_code == 200:
    root_gallery = ET.fromstring(response_gallery.content)

    data_list_gallery = []

    for item in root_gallery.findall('msgBody/placeList'):
        data_dict = {}
        try:
            data_dict['name'] = item.find('culName').text  # 이름 추출

        except AttributeError:
            print(f"Failed to find 'culName' in item: {ET.tostring(item)}")
            data_dict['name'] = None

        try:
            data_dict['latitude'] = float(item.find('gpsY').text)  # 위도 추출 ('gpsY')
            data_dict['longitude'] = float(item.find('gpsX').text)  # 경도 추출 ('gpsX')
        except (AttributeError, ValueError):
            print(f"Failed to find or convert latitude/longitude in item: {ET.tostring(item)}")
            continue

        data_list_gallery.append(data_dict)

    df_gallery = pd.DataFrame(data_list_gallery)

else:
    print(f'Request failed with status code {response.status_code}')

# 박물관 정보 가져오기
response_museum= requests.get(museum_url, params=params)

if response_museum.status_code == 200:
    root_museum= ET.fromstring(response_museum.content)

    data_list_museum= []

    for item in root_museum.findall('msgBody/placeList'):
        data_dict= {}
        try:
            data_dict['name']= item.find('culName').text

        except AttributeError:
            print(f"Failed to find 'culName' in item: {ET.tostring(item)}")
            data_dict['name']= None

        try:
          data_dict['latitude']= float(item.find('gpsY').text)
          data_dict['longitude']= float(item.find('gpsX').text)

        except (AttributeError, ValueError):
            print(f"Failed to find or convert latitude/longitude in item: {ET.tostring(item)}")
            continue

        data_list_museum.append(data_dict)

    df_museum= pd.DataFrame(data_list_museum)

else:
  	print(f'Request failed with status code {response.status_code}')


# 공연장 정보 가져오기
response_perform= requests.get(perform_url, params=params)

if response_perform.status_code == 200:
    root_perform= ET.fromstring(response_perform.content)

    data_list_perform= []

    for item in root_perform.findall('msgBody/placeList'):
        data_dict= {}
        try:
            data_dict['name']= item.find('culName').text

        except AttributeError:
            print(f"Failed to find 'culName' in item: {ET.tostring(item)}")
            data_dict['name']= None

        try:
          data_dict['latitude']= float(item.find('gpsY').text)
          data_dict['longitude']= float(item.find('gpsX').text)

        except (AttributeError, ValueError):
            print(f"Failed to find or convert latitude/longitude in item: {ET.tostring(item)}")
            continue

        data_list_perform.append(data_dict)

    df_perform= pd.DataFrame(data_list_perform)

else:
  	print(f'Request failed with status code {response.status_code}')



map_seoul= folium.Map(location=[37.56,126.97], zoom_start=12)

for idx in df_gallery.index:
  	if pd.notna(df_gallery.loc[idx,'name']) and pd.notna(df_gallery.loc[idx,'latitude']) and pd.notna(df_gallery.loc[idx,'longitude']):
    	  folium.Marker(
            location=[df_gallery.loc[idx,'latitude'], df_gallery.loc[idx,'longitude']],
            popup=df_gallery.loc[idx,'name'],
            icon=folium.Icon(icon='cloud')
        ).add_to(map_seoul)


for idx in df_museum.index:
  	if pd.notna(df_museum.loc[idx,'name']) and pd.notna(df_museum.loc[idx,'latitude']) and pd.notna(df_museum.loc[idx,'longitude']):
    	  folium.Marker(
            location=[df_museum.loc[idx,'latitude'], df_museum.loc[idx,'longitude']],
            popup=df_museum.loc[idx,'name'],
            icon=folium.Icon(color='red', icon='cloud')
       ).add_to(map_seoul)

for idx in df_perform.index:
  	if pd.notna(df_perform.loc[idx,'name']) and pd.notna(df_perform.loc[idx,'latitude']) and pd.notna(df_perform.loc[idx,'longitude']):
    	  folium.Marker(
            location=[df_perform.loc[idx,'latitude'], df_perform.loc[idx,'longitude']],
            popup=df_perform.loc[idx,'name'],
            icon=folium.Icon(color='green', icon='cloud')
       ).add_to(map_seoul)


map_seoul.save('seoul_art_and_museums.html')

map_seoul